In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import time

# 配置 Chrome 驅動
chrome_options = Options()
# chrome_options.add_argument("--headless")  # 隱藏瀏覽器
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# 這裡需要指定您的 ChromeDriver 的路徑
driver_path = 'chromedriver.exe'  # 替換為你的 chromedriver 路徑
service = Service(driver_path)

# 初始化 WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# 打開登入頁面
url = "https://app.yamol.tw/exam/24548"  # 修改為實際的登入 URL
driver.get(url)

# 點擊登入按鈕
login_button = driver.find_element(By.XPATH, "//a[contains(@class, 'MuiButton-containedPrimary') and contains(text(), '登入')]")
login_button.click()

# 增加額外的等待時間確保頁面加載完成
time.sleep(5)

# 填寫電子郵件和密碼並登錄
wait = WebDriverWait(driver, 20)  # 增加超時時間至 20 秒

# 填寫電子郵件
wait = WebDriverWait(driver, 10)
email_input = wait.until(EC.presence_of_element_located((By.NAME, 'email')))
email_input.send_keys("your_email@gmail.com")  # 用您的電子郵件替換

# 填寫密碼
password_input = driver.find_element(By.XPATH, "//input[@name='password']")
password_input.send_keys("your_password")  # 用您的密碼替換

# 點擊登入按鈕
login_button = driver.find_element(By.XPATH, "//button[@type='submit' and text()='登入']")
login_button.click()

# 等待頁面加載，這裡可以根據需要調整等待時間
time.sleep(5)


question_content = []  # 建立空的 list 儲存題目內容
options_content = []  # 建立空的 list 儲存選項內容
answer_content = []  # 建立空的 list 儲存答案內容
new_url_content = []  # 建立空的 list 儲存新 URL

visited_questions = set()  # 用來儲存已經抓取過的題目


# 點擊Index切換網頁內容
i = 1
while True:
    try:
        button = driver.find_element(By.XPATH, f"//button[contains(@class, 'text-info') and text()='{i}. ']")
        print(f"點擊 {i}. 題")
        i += 1
        button.click()
    except:
        print("已完成所有頁面")
        break

    # 等待新頁面加載完成
    time.sleep(2)

    # 用By.XPATH 定位到 QUESTION 1 並抓取內容
    stat_div = driver.find_element(By.XPATH, "//div[contains(text(), '統計：')]")
    statistics_text = stat_div.text
    print("統計內容：", statistics_text)

    match = re.search(r'#(\d+)', statistics_text)
    if match:
        seiunm = match.group(1)
        new_url = f"{url}?info=item.{seiunm}"  # 取得新的URL
        new_url_content.append(new_url)
        print("新的 URL:", new_url)

        if seiunm in visited_questions:
            print("已經抓取過該題目")
            continue

        visited_questions.add(seiunm)

        # 點擊另一個題目頁面
        driver.get(new_url)
        time.sleep(2)

        # 定位到 題目欄位 並抓取內容
        question_div = driver.find_element(By.CLASS_NAME, 'item-htmlRender')
        inner_divs = question_div.find_element(By.CLASS_NAME, 'MuiBox-root')
        question_content.append(inner_divs.text)

        # 定位到 選項欄位 並抓取內容
        options_div = driver.find_element(By.ID, f'item-{seiunm}-options')
        inner_divs = options_div.find_elements(By.CLASS_NAME, 'MuiBox-root')
        options = [div.text for div in inner_divs]
        options_content.append(options)

        # 定位到 答案欄位 並抓取內容
        try:
            answer_div = driver.find_element(By.XPATH, "//div[contains(text(), '答案：')]/a")
            answer_letter = answer_div.text
        except:
            answer_letter = "登入後查看"  # 如果答案無法取得
        answer_content.append(answer_letter)

        print(f"問題: {question_content[-1]}")
        print(f"選項: {options}")
        print(f"答案: {answer_letter}")

# 將題目、選項、答案加入 DataFrame
df = pd.DataFrame(columns=['題目', 'A', 'B', 'C', 'D', 'E', '答案', '題目代碼'])

for i in range(len(question_content)):
    options = options_content[i] + [""] * (5 - len(options_content[i]))  # 補足選項至5個
    df.loc[i] = [
        question_content[i],
        options[0],
        options[1],
        options[2],
        options[3],
        options[4],
        answer_content[i],
        new_url_content[i]
    ]

df.to_csv('python.csv', index=False) # 儲存成 CSV 檔案

點擊 1. 題
統計內容： 統計：A(2), B(1), C(2), D(0), E(0) #912563
新的 URL: https://app.yamol.tw/exam/24548?info=item.912563
問題: QUESTION 1 Examine this package: CREATE OR REPLACE PACKAGE BB_PACK IS V_MAX_TEAM_SALARY NUMBER(12,2); PROCEDURE ADD_PLAYER(V_ID IN NUMBER, V_LAST_NAME VARCHAR2, NUMBER); END BB_PACK; / CREATE OR REPLACE PACKAGE BODY BB_PACK IS PROCEDURE UPD_PLAYER_STAT (V_ID IN NUMBER, V_AB IN NUMBER DEFAULT 4, V_HITS IN NUMBER) IS BEGIN UPDATE PLAYER_BAT_STAT SET AT_BATS = AT_BATS + V_AB, HITS = HITS + V_HITS WHERE PLAYER_ID = V_ID; COMMIT; END UPD_PLAYER_STAT; PROCEDURE ADD_PLAYER (V_ID IN NUMBER, V_LAST_NAME VARCHAR2, V_SALARY NUMBER) IS BEGIN INSERT INTO PLAYER(ID,LAST_NAME,SALARY) VALUES (V_ID, V_LAST_NAME, V_SALARY); UPD_PLAYER_STAT(V_ID,0,0); END ADD_PLAYER; END BB_PACK; You make a change to the body of the BB_PACK package. The BB_PACK body is recompiled. What happens if the stand alone procedure VALIDATE_PLAYER_STAT references this package?
選項: ['(A) VALIDATE_PLAYER_STAT cannot rec